## Pattern Match
    
    为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。
    
    例如，定义一个目标: "I want X"， 我们可以表示成 "I want ?X", 意思就是?X是一个用来占位的符号。
    
    如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [1]:
import random

In [2]:
# 参数：pat: string
# 方法：判断是不是变量, isalpha() 方法检测字符串是否只由字母组成, all表示所有为True返回True否则返回False
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:]) 

# 参数：pattern:list(str), saying:list(str)
# 方法：saying 和 模板 pattern 进行匹配，匹配每一个字符，
# 如果字符相等继续比较下一个字符，如果不等直接返回 False，如果全部相等返回True，如果字符 is_variable 直接 return True
def pat_match_simple(pattern, saying):
    res = {}
    for i in range(len(pattern)):
        if is_variable(pattern[i]):
            res[pattern[i]] = saying[i]
            continue
        if pattern[i] != saying[i]: return None
    return res

# 参数：pattern：string；rule：dict(k=string, v=string)
# 方法：根据 rule 规则，把 pattern 补充完成
def supplement_word(pattern, rule):
    return ' '.join(s for s in [rule[word] if word in rule.keys() else word for word in pattern.split()])

# 参数：根据saying，选择 rules 最合适的回答
# 方法：返回最合适的句子
def get_response_1(saying, rules):
    for k,v in rules.items():
        match_rule = pat_match_simple(k.split(), saying.split())
        if match_rule: return supplement_word(random.choice(v), match_rule) 
    return saying

In [3]:
s_1 = supplement_word('?X plus ?Y equal 5', pat_match_simple("?X greater than ?Y".split(),"3 greater than 2".split()))
print(s_1)

3 plus 2 equal 5


In [4]:
defined_rules = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}
ans_1 = get_response_1('I need iphone', defined_rules)
print(ans_1)

Why do you need iphone ?


In [5]:
ans_2 = get_response_1('My mom told me something', defined_rules)
print(ans_2)

How do you think about your mom ?


In [6]:
input_str = ''
while True: 
    input_str = input("Input:")
    if input_str == '0': break # 输入0退出
    print('Response:'+str(get_response_1(input_str, defined_rules)))

Input:My father told me something
Response:Talk about more about your father
Input:0



### Segment Match
    我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的
    
    "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？

    为了解决这个问题，我们可以新建一个变量类型 "?*X", 这种类型多了一个星号(*),表示匹配多个

    首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [7]:
# 参数解释： pattern_str: string; saying_str: string
# 方法：用于找出占位符的值，返回字典类型
def pat_match(pattern_str, saying_str):
    pattern = pattern_str.replace('?*', '?').split()
    saying = saying_str.split()
    res = {}
    i = j = 0
    key = -1
    n = 0
    for e in pattern:
        if is_variable(e):n += 1
    while i < len(pattern) and j < len(saying):
        if is_variable(pattern[i]): 
            key = i
            i += 1
            continue
        if pattern[i] == saying[j]:
            if pattern[key] not in res.keys() and key != -1:
                res[pattern[key]] = saying[key:j]
            i += 1
        j += 1
    if j != len(saying):res[pattern[key]] = saying[j:]
    if len(res) != n: return {}
    return res

# 参数说明：pattern: string 
# 方法：判断是不是一个占位符
def is_variable(pattern):
    return pattern.startswith('?') and all(a.isalpha() for a in pattern[1:])

# 参数说明: pattern: 待补充的字符串；rule(string:list): 是占位符字典
# 方法：根据rule映射关系，补充 pattern
def supplement_sentence(pattern, rule):
    return ' '.join(s for s in [' '.join(rule[word]) if word in rule.keys() else word 
                                for word in pattern.replace('?*', '?').split()])

### 问题1
    
    编写一个程序, get_response(saying, response_rules)输入是一个字符串 + 我们定义的 rules.

In [8]:
# 参数说明: saying:string, response_rules:dict(k=string:v=list(string))
# 方法：根据saying，返回合适的句子
def get_response(saying, response_rules):
    for k,v in response_rules.items():
        d_match = pat_match(k, saying)
        if d_match: 
            return supplement_sentence(random.choice(v), d_match)
    return saying

In [9]:
dict_1 = pat_match('?P is very good and ?*X', "my dog is very good and my cat is very cute")
print(dict_1)

{'?P': ['my', 'dog'], '?X': ['my', 'cat', 'is', 'very', 'cute']}


In [10]:
dict_2 = pat_match('?*x I want ?*y', "I want gift")
print(dict_2)

{'?x': [], '?y': ['gift']}


In [11]:
sentence_1 = supplement_sentence("Why do you neeed ?X", pat_match('I need ?*X', "I need an iPhone"))
print(sentence_1)

Why do you neeed an iPhone


In [12]:
sentence_2 = supplement_sentence("what would it mean if you got ?y", pat_match('?*x I want ?*y', "I want gift"))
print(sentence_2)

what would it mean if you got gift


In [13]:
rules = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y',
                   'Do you wish that ?y', 
                   'What do you think about ?y',
                   'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
}
ans_str = get_response('I want the gift', rules)
print(ans_str)

Suppose you got the gift soon


In [14]:
input_str = ''
while True: # 输入0退出
    input_str = input("Input:")
    if input_str == '0': break
    print('Response:',get_response(input_str, rules))

Input:James no I am boy
Response: Are you saying 'No' just to be negative?
Input:0


## 改写以上程序，将程序变成能够支持中文输入的模式。

In [15]:
import random

In [16]:
# 参数: pattern:string
# 方法: 判断是否为占位符
def is_variable(pattern):
    return pattern.startswith('?')

# 参数: pattern_str:string, saying_str:string
# 方法: 统计占位符对应的值dict(k=string,v=string)
def pat_match_ch(pattern_str, saying_str):
    pattern_str = pattern_str.replace('?*', '?')
    res = {}
    i = j = k = 0
    key = -1
    while i < len(pattern_str) and j < len(saying_str):
        if is_variable(pattern_str[i]) and len(pattern_str[i:i + 2]) == 2:
            key = i
            i += 2
            k = j
            continue
        if pattern_str[i] == saying_str[j]:
            if pattern_str[key:key + 2] not in res.keys() and key != '':
                res[pattern_str[key:key + 2]] = saying_str[k:j]
            i += 1
        j += 1
    if j == len(saying_str) and i < len(pattern_str):return {}
    if i == len(pattern_str) and j < len(saying_str):
        if is_variable(pattern_str[i-2]) and len(pattern_str[i-2:i]) == 2:
            res[pattern_str[i-2:i]] = saying_str[j:]
        else:
            return {}
    return res

# 参数说明: pattern: 待补充的字符串；rule(string:list): 是占位符字典
# 方法：根据rule映射关系，补充 pattern
def supplement_sentence_ch(pattern, rule):
    pattern = pattern.replace('?*','?')
    res = ''
    i = 0
    while i < len(pattern):
        if is_variable(pattern[i]) and len(pattern[i:i+2])==2:
            res += rule[pattern[i:i+2]]
            i += 2
        else:
            res += pattern[i]
            i += 1
    return res

### 问题2

    改写以上程序，将程序变成能够支持中文输入的模式。 提示: 你可以需用用到 jieba 分词
    
    但是，我没有用 jieba 分词，因为觉得 jieba 分词并不能完全找出占位符的值.
    （例如：‘我是真的好想和你在一起’, ‘我是?*x的好?*y和你在?*z’起）

In [17]:
# 参数说明: saying:string, response_rules:dict(k=string:v=list(string))
# 方法：根据saying，返回合适的句子
def get_response_ch(saying, response_rules):
    for k,v in response_rules.items():
        d_match = pat_match_ch(k, saying)
        if d_match: 
            return supplement_sentence_ch(random.choice(v), d_match)
    return saying

In [18]:
sentence_1 = supplement_sentence_ch("你觉得?z有什么问题吗?", pat_match_ch('?*x和?*y一样?*z', "你和猪一样蠢死啦"))
print(sentence_1)

你觉得蠢死啦有什么问题吗?


In [19]:
rules_ch = {
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', 
                  '为什么你想?y', 
                  '?x觉得... 你可以想想你很快就可以有?y了', 
                  '你看?x像?y不', 
                  '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', 
                 '嗯嗯，多说一点和你家里有关系的',
                 '他对你影响很大吗？', 
                 '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [20]:
for _ in range(5):
    ans_str = get_response_ch('我不要苹果', rules_ch)
    print(ans_str)

你说 不，是想表达不想的意思吗？
你有一点负能量
你说 不，是想表达不想的意思吗？
为什么不？
为什么不？


### 问题3

    多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

In [21]:
input_str = ''
while True: # 输入0退出
    input_str = input("Input:")
    if input_str == '0':break
    print('Response:',get_response_ch(input_str, rules_ch))

Input:某些人太狡猾了
Response: 具体是哪些人呢?
Input:我愿意嫁给你
Response: 我可以帮你嫁给你吗？
Input:我爸爸是警察
Response: 他对你影响很大吗？
Input:我爸爸是警察
Response: 他对你影响很大吗？
Input:我爸爸是警察
Response: 每当你想起你爸爸的时候， 你还会想起其他的吗?
Input:我妈妈是老师
Response: 她对你影响很大吗？
Input:我妈妈是教师
Response: 嗯嗯，多说一点和你家里有关系的
Input:你觉得他们是二货吗
Response: 你为什么会对我是不是二货感兴趣?
Input:每个人都有自己的生活
Response: 你看到的其实只是一小部分人
Input:有些人真的很善良
Response: 具体是哪些人呢?
Input:你为什么不喜欢我
Response: 我有什么好的呢？
Input:你为什么不喜欢我
Response: 我有什么好的呢？
Input:你为什么不喜欢我
Response: 你想要我吗？
Input:你为什么不喜欢我
Response: 喜欢我的哪里？
Input:你为什么不喜欢我
Response: 我有什么好的呢？
Input:你为什么不喜欢我
Response: 我有什么好的呢？
Input:你为什么不喜欢我
Response: 我有什么好的呢？
Input:你为什么不喜欢我
Response: 喜欢我的哪里？
Input:你为什么不喜欢我
Response: 我有什么好的呢？
Input:0


### 问题4

    1、这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？
    Ans: 优点：原理简单、易理解；缺点：需要根据模板进行回答、还不够智能；改进：能根据词语自动组成适合于该场景下的句子就好了
    2、什么是数据驱动？数据驱动在这个程序里如何体现？
    Ans: 数据驱动就是一切规律和结论都是经过数据模型来体现。在程序里面通过各种数值运算来体现。
    3、数据驱动与 AI 的关系是什么？
    Ans: AI是以海量数据为基础发展而来的。